In [1]:
!pip install elasticsearch

In [2]:
from elasticsearch import Elasticsearch
from datetime import datetime

In [3]:
def connect():
    elastic = Elasticsearch("http://localhost:9200")

    # Successful response!
    print(elastic.info())

    if not elastic.ping():
        print('Could not connect to Elastic Search!')
    else:
        print("Connected to elastic engine!")
        return elastic

In [4]:
def create_index(elastic, name="", body={}):
    if elastic.indices.exists(name):
        print("Index with the given name already exists!")
        return -1
    else:
        body = {
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
        }
        response = elastic.indices.create(index=name, ignore=400, body=body)
        print(response)
        return 1

In [5]:
elastic = connect()

{'name': '15c6715ee9b5', 'cluster_name': 'docker-cluster', 'cluster_uuid': '2l4BsHlgTKG8jIPsQ6ULUg', 'version': {'number': '7.8.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'b5ca9c58fb664ca8bf9e4057fc229b3396bf3a89', 'build_date': '2020-07-21T16:40:44.668009Z', 'build_snapshot': False, 'lucene_version': '8.5.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}
Connected to elastic engine!


In [8]:
# Create an Index
create_index(elastic, "capitals")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'capitals'}


1

In [7]:
elastic.indices.delete(index='capitals', ignore=[400, 404])


{'acknowledged': True}

In [9]:
# Create an Index
create_index(elastic, "tests")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'tests'}


1

In [10]:
# Delete an index
elastic.indices.delete(index="tests")

{'acknowledged': True}

In [12]:
elastic.indices.exists(index="tests")

False

In [13]:
# Inserting data
data_1 = {"country": "India", "capital": "Delhi", "text": "Delh is the capital of India"}
data_2 = {"country": "USA", "capital": "Washington DC", "text": "Washington DC is the capital of USA"}
data_3 = {"country": "Belgium", "capital": "Brussel", "text": "Brussel is the capital of Belgium"}
data_4 = {"country": "Canada", "capital": "Ottawa", "text": "Ottawa is the capital of Canada"}

In [14]:
elastic.index(index="capitals", doc_type="Places", body=data_1)

c:\Users\UIF13879\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


{'_index': 'capitals',
 '_type': 'Places',
 '_id': 'TTiKRIQBhfFNwNqecdcn',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [15]:
elastic.index(index="capitals", doc_type="Places", body=data_2)

{'_index': 'capitals',
 '_type': 'Places',
 '_id': 'UjiKRIQBhfFNwNqem9dl',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

In [16]:
elastic.index(index="capitals", doc_type="Places", body=data_3)

{'_index': 'capitals',
 '_type': 'Places',
 '_id': 'VDiKRIQBhfFNwNqep9cR',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 2,
 '_primary_term': 1}

In [17]:
elastic.index(index="capitals", doc_type="Places", body=data_4)

{'_index': 'capitals',
 '_type': 'Places',
 '_id': 'VjiKRIQBhfFNwNqerNfF',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 3,
 '_primary_term': 1}

In [18]:
elastic.update(index='capitals',doc_type='Places',id="TTiKRIQBhfFNwNqecdcn",
                body={
                    "doc": {"country": "India", "capital": "Delhi", "text": "Delhi is the capital of India"}, 
})

c:\Users\UIF13879\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in document update requests is deprecated, use the endpoint /{index}/_update/{id} instead.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


{'_index': 'capitals',
 '_type': 'Places',
 '_id': 'TTiKRIQBhfFNwNqecdcn',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 4,
 '_primary_term': 1}

In [19]:
result = elastic.search(
    index="capitals",
    body={
        "query": {
            "match_all": {}
        }
    }
)
print(len(result['hits']['hits']))

4


In [20]:
elastic.update(index='capitals',doc_type='Places',id="10",
                body={
                    "doc": {"country": "India", "capital": "Delhi", "text": "Delhi is the capital of India!"}, 
                    "doc_as_upsert": True
})

{'_index': 'capitals',
 '_type': 'Places',
 '_id': '10',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 5,
 '_primary_term': 1}

In [21]:
result = elastic.search(
    index="capitals",
    body={
        "query": {
            "match_all": {}
        }
    }
)
print(len(result['hits']['hits']))

5


In [22]:
body = {
    "from": 0,
    "size": 0,
    "query": {
        "match": {
            "capital": "Delhi"
        }
    }
}
elastic.search(index="capitals", body=body)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [23]:
body = {
    "from": 0,
    "size": 5,
    "query": {
        "match": {
            "capital": "Delhi"
        }
    }
}
elastic.search(index="capitals", body=body)

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 0.7361701,
  'hits': [{'_index': 'capitals',
    '_type': 'Places',
    '_id': 'TTiKRIQBhfFNwNqecdcn',
    '_score': 0.7361701,
    '_source': {'country': 'India',
     'capital': 'Delhi',
     'text': 'Delhi is the capital of India'}},
   {'_index': 'capitals',
    '_type': 'Places',
    '_id': '10',
    '_score': 0.7361701,
    '_source': {'country': 'India',
     'capital': 'Delhi',
     'text': 'Delhi is the capital of India!'}}]}}

In [24]:
body = {
    "from": 0,
    "size": 5,
    "query": {
        "match": {
            "text": "Delhi is the capital of "
        }
    }
}
elastic.search(index="capitals", body=body)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
  'max_score': 1.3408768,
  'hits': [{'_index': 'capitals',
    '_type': 'Places',
    '_id': 'TTiKRIQBhfFNwNqecdcn',
    '_score': 1.3408768,
    '_source': {'country': 'India',
     'capital': 'Delhi',
     'text': 'Delhi is the capital of India'}},
   {'_index': 'capitals',
    '_type': 'Places',
    '_id': '10',
    '_score': 1.3408768,
    '_source': {'country': 'India',
     'capital': 'Delhi',
     'text': 'Delhi is the capital of India!'}},
   {'_index': 'capitals',
    '_type': 'Places',
    '_id': 'VDiKRIQBhfFNwNqep9cR',
    '_score': 0.29974607,
    '_source': {'country': 'Belgium',
     'capital': 'Brussel',
     'text': 'Brussel is the capital of Belgium'}},
   {'_index': 'capitals',
    '_type': 'Places',
    '_id': 'VjiKRIQBhfFNwNqerNfF',
    '_score': 0.29974607,
    '_source': {'country': 'Canada',
     'capital': 'Ot

In [25]:
body = {
    "from": 0,
    "size": 5,
    "query": {
        "match_phrase": {
            "text": "the capital of "
        }
    }
}
elastic.search(index="capitals", body=body)

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
  'max_score': 0.22480953,
  'hits': [{'_index': 'capitals',
    '_type': 'Places',
    '_id': 'VDiKRIQBhfFNwNqep9cR',
    '_score': 0.22480953,
    '_source': {'country': 'Belgium',
     'capital': 'Brussel',
     'text': 'Brussel is the capital of Belgium'}},
   {'_index': 'capitals',
    '_type': 'Places',
    '_id': 'VjiKRIQBhfFNwNqerNfF',
    '_score': 0.22480953,
    '_source': {'country': 'Canada',
     'capital': 'Ottawa',
     'text': 'Ottawa is the capital of Canada'}},
   {'_index': 'capitals',
    '_type': 'Places',
    '_id': 'TTiKRIQBhfFNwNqecdcn',
    '_score': 0.22480953,
    '_source': {'country': 'India',
     'capital': 'Delhi',
     'text': 'Delhi is the capital of India'}},
   {'_index': 'capitals',
    '_type': 'Places',
    '_id': '10',
    '_score': 0.22480953,
    '_source': {'country': 'India',
     'capital'

In [27]:
body = {
    "from": 0,
    "size": 5,
    "query": {
        "bool": {
            "must_not": {
                'match': {
                    "capital": "Ottawa"
                }
            },
        "should": {
            "match": {
                "text": "is the capital of"
        }
        }
        }
    }
}
res = elastic.search(index="capitals", body=body)

In [28]:
print(res['hits']['hits'])

[{'_index': 'capitals', '_type': 'Places', '_id': 'VDiKRIQBhfFNwNqep9cR', '_score': 0.29974607, '_source': {'country': 'Belgium', 'capital': 'Brussel', 'text': 'Brussel is the capital of Belgium'}}, {'_index': 'capitals', '_type': 'Places', '_id': 'TTiKRIQBhfFNwNqecdcn', '_score': 0.29974607, '_source': {'country': 'India', 'capital': 'Delhi', 'text': 'Delhi is the capital of India'}}, {'_index': 'capitals', '_type': 'Places', '_id': '10', '_score': 0.29974607, '_source': {'country': 'India', 'capital': 'Delhi', 'text': 'Delhi is the capital of India!'}}, {'_index': 'capitals', '_type': 'Places', '_id': 'UjiKRIQBhfFNwNqem9dl', '_score': 0.28090286, '_source': {'country': 'USA', 'capital': 'Washington DC', 'text': 'Washington DC is the capital of USA'}}]


In [29]:
body = {
    "from": 0,
    "size": 5,
    "query": {
        "regexp": {
                "country": ".*"
        }
    }
}
elastic.search(index="capitals", body=body)

{'took': 11,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'capitals',
    '_type': 'Places',
    '_id': 'UjiKRIQBhfFNwNqem9dl',
    '_score': 1.0,
    '_source': {'country': 'USA',
     'capital': 'Washington DC',
     'text': 'Washington DC is the capital of USA'}},
   {'_index': 'capitals',
    '_type': 'Places',
    '_id': 'VDiKRIQBhfFNwNqep9cR',
    '_score': 1.0,
    '_source': {'country': 'Belgium',
     'capital': 'Brussel',
     'text': 'Brussel is the capital of Belgium'}},
   {'_index': 'capitals',
    '_type': 'Places',
    '_id': 'VjiKRIQBhfFNwNqerNfF',
    '_score': 1.0,
    '_source': {'country': 'Canada',
     'capital': 'Ottawa',
     'text': 'Ottawa is the capital of Canada'}},
   {'_index': 'capitals',
    '_type': 'Places',
    '_id': 'TTiKRIQBhfFNwNqecdcn',
    '_score': 1.0,
    '_source': {'country': 'India',
     'capital': 'D